In [ ]:
%matplotlib widget

import sys
import os
import time
import math as math
from datetime import datetime
import pickle 
import numpy as np
import astropy as ast
from astropy.io import fits
import astropy.io.fits as pyfits
from astropy.table import Table, hstack, vstack, Column
from astropy.stats import sigma_clipped_stats
from astropy.coordinates import Angle
from astropy.time import Time
from photutils.aperture import EllipticalAnnulus, aperture_photometry
import matplotlib
import matplotlib.pyplot as plt
import scipy as sp
import scipy.integrate
import scipy.constants
import scipy.special
import scipy.misc
from scipy import integrate
from scipy.optimize import curve_fit
import scipy.interpolate
from scipy.interpolate import CubicSpline
import glob2
import shutil
from IPython.display import clear_output
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import axes3d
from tqdm.notebook import tqdm as tqdm

import cv2

import Utility3_4 as ut

pi = sp.constants.pi
k= sp.constants.k
q_e = sp.constants.e

DimY = 1096
DimX = 1936

Offset = 2 #ADU Blacklevel Offset

#Gain = 2.421
#SigmaGain = 0.243
Gain = 2.398
SigmaGain = 0.069

DisectString = '\1/'
BackSlash = DisectString[0]
ForwardSlash = '/'

In [ ]:
#Loading in Master Hots Array
HotsArray = np.loadtxt('SteadyStateDataSets/MasterHotsArray.csv', delimiter=',')

SetNamesList = ['X0_Y0','X0_YM1','X0_YM2','X0_YM3','X0_YP1','X0_YP2','X0_YP3','XM2_Y0','XM2_YM1','XM2_YP1','XP2_Y0','XP2_YM1','XP2_YP1','XP4_Y0']

CoordsDict = {}
FluxFrameDict = {}
ErrFluxFrameDict = {}

#Populating Dictionaries
for SetName in tqdm(SetNamesList):
    #making flux frame and error flux frame dictionaries
    LoadDirectory = 'SteadyStateDataSets/' + SetName + '/'
    #MeanFluxFrame = np.loadtxt(LoadDirectory + 'PrelimMeanFluxFrame' + '.csv', delimiter=',')
    #ErrorMeanFluxFrame = np.loadtxt(LoadDirectory + 'PrelimErrorMeanFluxFrame' + '.csv', delimiter=',')
    #MeanFluxFrame = np.loadtxt(LoadDirectory + 'PrelimMeanFluxFrame2' + '.csv', delimiter=',')
    #ErrorMeanFluxFrame = np.loadtxt(LoadDirectory + 'PrelimErrorMeanFluxFrame2' + '.csv', delimiter=',')
    MeanFluxFrame = np.loadtxt(LoadDirectory + 'PrelimDarkSubMeanFluxFrame4' + '.csv', delimiter=',')
    ErrorMeanFluxFrame = np.loadtxt(LoadDirectory + 'PrelimDarkSubErrorMeanFluxFrame4' + '.csv', delimiter=',')
    
    FluxFrameDict.update({SetName:MeanFluxFrame})
    ErrFluxFrameDict.update({SetName:ErrorMeanFluxFrame})
    
    #making mosaic coordinates dictionary
    x_y = []
    SM1 = 'X'
    for s in SetName:
        if s.isdigit():
            if SM1 == 'P' or s == '0':
                Sign = 1
            if SM1 == 'M':
                Sign = -1
            x_y.append(int(Sign*int(s)))
        if s != 'X':
            SM1 = s
    CoordsDict.update({SetName:np.array(x_y)})
    
#Loading 1st test translation dictionaries
with open('TranslateX_Dict.pkl', 'rb') as f:
    TranslateX_Dict_Test1 = pickle.load(f)
        
with open('TranslateY_Dict.pkl', 'rb') as f:
    TranslateY_Dict_Test1 = pickle.load(f)
    
#Loading in the 1st test (yes, 1st even though it says Test2) gradient descent translation dictionaries
with open('TranslateX_Dict_GD_Test2.pkl', 'rb') as f:
    TranslateX_Dict_GD_Test2 = pickle.load(f)
        
with open('TranslateY_Dict_GD_Test2.pkl', 'rb') as f:
    TranslateY_Dict_GD_Test2 = pickle.load(f)
    
#Loading in the 2st test
with open('TranslateX_Dict_GD2.pkl', 'rb') as f:
    TranslateX_Dict_GD2 = pickle.load(f)
        
with open('TranslateY_Dict_GD2.pkl', 'rb') as f:
    TranslateY_Dict_GD2 = pickle.load(f)

In [ ]:
#Starting the 4th Test Dictionary

TranslateX_Dict_GD4 = {}
TranslateY_Dict_GD4 = {}

Coords = -1*CoordsDict['X0_Y0']
MosaicX = Coords[0]
MosaicY = Coords[1]

TranslateX_Dict_GD4.update({'X0_Y0':int(((MosaicX/2) + 2)*DimX)})
TranslateY_Dict_GD4.update({'X0_Y0':int((MosaicY + 3)*DimY)})

with open('TranslateX_Dict_GD4.pkl', 'wb') as f:
    pickle.dump(TranslateX_Dict_GD4, f)
        
with open('TranslateY_Dict_GD4.pkl', 'wb') as f:
    pickle.dump(TranslateY_Dict_GD4, f)

#Loading in 3rd Test Dictionary
with open('TranslateX_Dict_GD4.pkl', 'rb') as f:
    TranslateX_Dict_GD4 = pickle.load(f)
        
with open('TranslateY_Dict_GD4.pkl', 'rb') as f:
    TranslateY_Dict_GD4 = pickle.load(f)

In [ ]:
def ROItoPoints(ROI):
    
    x1 = ROI[0]
    y1 = ROI[2]

    x2 = ROI[1]
    y2 = ROI[2]
    
    x3 = ROI[1]
    y3 = ROI[3]
    
    x4 = ROI[0]
    y4 = ROI[3]
    
    x = [x1,x2,x3,x4]
    y = [y1,y2,y3,y4]
    
    return x, y

#Get dx and dy from translate dict
def Recall_dx_dy(SetName1,SetName2,TranslateX_Dict,TranslateY_Dict):
    
    TranslateX0 = TranslateX_Dict[SetName1]
    TranslateY0 = TranslateY_Dict[SetName1]
    
    dx = TranslateX_Dict[SetName2] - TranslateX0
    dy = TranslateY_Dict[SetName2] - TranslateY0
    
    return dx, dy

def CilpAndMake8bit(img,clip_value):
    img_clip = np.clip(img, 0, clip_value)
    img_stretch = np.rint((img_clip/clip_value)*256)
    img_clip = np.clip(img_stretch, 0, 255)
    img_8bit = img_clip.astype('uint8')
    return img_8bit

def StripClipAndMake8bit(img,clip_min,clip_max):
    img_clip = np.clip(img, 0, clip_max)
    img_clip = img_clip - clip_min
    img_stretch = np.rint((img_clip/(clip_max-clip_min))*256)
    img_clip = np.clip(img_stretch, 0, 255)
    img_8bit = img_clip.astype('uint8')
    return img_8bit

def HelpPickROI(SetName1,SetName2,ROI1,HotsArray,dx,dy):
    
    dx = round(dx)
    dy = round(dy)
    
    MeanFluxFrame_1 = FluxFrameDict[SetName1]
    img1 = np.zeros((DimY,DimX))
    img1 =  MeanFluxFrame_1
    img1 = np.where(HotsArray > 0, 0, img1)
    
    MeanFluxFrame_2 = FluxFrameDict[SetName2]
    img2 = np.zeros((DimY,DimX))
    img2 =  MeanFluxFrame_2
    img2 = np.where(HotsArray > 0, 0, img2)
    
    OverlapFrame1 = np.zeros((DimY,DimX))
    OverlapFrame2 = np.zeros((DimY,DimX))
    OverlapHotsArray = np.zeros((DimY,DimX))
    OverlapHotsArray = HotsArray
    
    for xi in range(DimX):
        for yi in range(DimY):
            
            NewXi = xi + dx
            NewYi = yi + dy
            
            #if NewXi < DimX and NewYi < DimY:
            if NewXi < DimX and NewYi < DimY and NewXi > -1 and NewYi > -1:
                
                OverlapFrame1[NewYi,NewXi] = img1[NewYi,NewXi]
                OverlapFrame2[NewYi,NewXi] = img2[yi,xi]
                
                OverlapHotsArray[NewYi,NewXi] = HotsArray[yi,xi]
                
    OverlapFrame1 = np.where(OverlapHotsArray > 0, 0, OverlapFrame1)
    OverlapFrame2 = np.where(OverlapHotsArray > 0, 0, OverlapFrame2)
    
    #ROI2 = [ROI1[0]+dx,ROI1[1]+dx,ROI1[2]+dy,ROI1[3]+dy]
    #ROI2 = [DimX - ROI1[0]+dx,DimX - ROI1[1]+dx,DimY - ROI1[2]+dy,DimY - ROI1[3]+dy]
    ROI2 = [ROI1[0]-dx,ROI1[1]-dx,ROI1[2]-dy,ROI1[3]-dy]
    #ROI2 isn't being calculated properly
    

    xROI1, yROI1 = ROItoPoints(ROI1)
    xROI2, yROI2 = ROItoPoints(ROI2)
    
    matplotlib.rcParams.update(matplotlib.rcParamsDefault)

    f, ax = plt.subplots(4, 1)
    #im1 = ax[0].imshow(OverlapFrame1, cmap='turbo', vmin = 0, vmax = 5)
    im1 = ax[0].imshow(img1, cmap='turbo', vmin = 0, vmax = 5)
    divider = make_axes_locatable(ax[0])
    cax1 = divider.append_axes("right", size="5%", pad=0.15)
    cbar1 = plt.colorbar(im1, cax=cax1)
    cbar1.formatter.set_powerlimits((0, 0))
    ax[0].fill(xROI1, yROI1, facecolor='none', edgecolor='magenta', linewidth=2, label = "ROI1")
    #ax[0].fill(xROI2, yROI2, facecolor='none', edgecolor='magenta', linewidth=2, label = "ROI1")
    ax[0].set_xlabel('Pixel x')
    ax[0].set_ylabel('Pixel y')
    ax[0].set_title(r"Frame 1 with ROI [$e^{-}/s$]"
                   "\n" r"490.5 eV", loc = 'center')

    #im2 = ax[1].imshow(OverlapFrame2, cmap='turbo', vmin = 0, vmax = 5)
    im2 = ax[1].imshow(img2, cmap='turbo', vmin = 0, vmax = 5)
    divider = make_axes_locatable(ax[1])
    cax2 = divider.append_axes("right", size="5%", pad=0.15)
    cbar2 = plt.colorbar(im2, cax=cax2)
    cbar2.formatter.set_powerlimits((0, 0))
    ax[1].fill(xROI2, yROI2, facecolor='none', edgecolor='magenta', linewidth=2, label = "ROI2")
    #ax[1].fill(xROI1, yROI1, facecolor='none', edgecolor='magenta', linewidth=2, label = "ROI2")
    ax[1].set_xlabel('Pixel x')
    ax[1].set_ylabel('Pixel y')
    ax[1].set_title(r"Frame 2with ROI [$e^{-}/s$]"
                   "\n" r"490.5 eV", loc = 'center')
    
    im3 = ax[2].imshow(OverlapFrame1, cmap='turbo', vmin = 0, vmax = 5)
    #im1 = ax[0].imshow(img1, cmap='turbo', vmin = 0, vmax = 5)
    divider = make_axes_locatable(ax[2])
    cax3 = divider.append_axes("right", size="5%", pad=0.15)
    cbar3 = plt.colorbar(im3, cax=cax3)
    cbar3.formatter.set_powerlimits((0, 0))
    ax[2].fill(xROI1, yROI1, facecolor='none', edgecolor='magenta', linewidth=2, label = "ROI1")
    #ax[0].fill(xROI2, yROI2, facecolor='none', edgecolor='magenta', linewidth=2, label = "ROI1")
    ax[2].set_xlabel('Pixel x')
    ax[2].set_ylabel('Pixel y')
    ax[2].set_title(r"Frame 1 Overlap with ROI [$e^{-}/s$]"
                   "\n" r"490.5 eV", loc = 'center')

    im4 = ax[3].imshow(OverlapFrame2, cmap='turbo', vmin = 0, vmax = 5)
    #im2 = ax[1].imshow(img2, cmap='turbo', vmin = 0, vmax = 5)
    divider = make_axes_locatable(ax[3])
    cax4 = divider.append_axes("right", size="5%", pad=0.15)
    cbar4 = plt.colorbar(im4, cax=cax4)
    cbar4.formatter.set_powerlimits((0, 0))
    #ax[1].fill(xROI2, yROI2, facecolor='none', edgecolor='magenta', linewidth=2, label = "ROI2")
    ax[3].fill(xROI1, yROI1, facecolor='none', edgecolor='magenta', linewidth=2, label = "ROI2")
    ax[3].set_xlabel('Pixel x')
    ax[3].set_ylabel('Pixel y')
    ax[3].set_title(r"Frame 2 Overlap with ROI [$e^{-}/s$]"
                   "\n" r"490.5 eV", loc = 'center')

    plt.subplots_adjust(hspace = 0.35)
    figure = plt.gcf()
    figure.set_size_inches(10, 20)
    plt.show()
        
    if ROI2[0] > -1 and ROI2[1] < DimX + 1 and ROI2[2] > -1 and ROI2[3] < DimY + 1:
        print("Valid combination of dx, dy, and ROI1.")
        print("ROI1:")
        print(ROI1)
        print("ROI2:")
        print(ROI2)
    else:
        print("Invalid combination of dx, dy, and ROI1.")
        print("ROI1:")
        print(ROI1)
        print("ROI2:")
        print(ROI2)

In [ ]:
def WeightedQualityFunction(SetName1,SetName2,ROI1,HotsArray,dx,dy,plot1=False,plot2=False,plotResiduals=False,SavePlots=False):
    #Returns the error weighted mean per pixel absolute residual in the ROI given a dx and dy and the ROI in the first image set
    
    dx = round(dx)
    dy = round(dy)
    
    #Making the two images given the set names and zeroing out the hotpixels
    MeanFluxFrame_1 = FluxFrameDict[SetName1]
    img1 = np.zeros((DimY,DimX))
    img1 =  MeanFluxFrame_1
    img1 = np.where(HotsArray > 0, 0, img1)
    
    ErrMeanFluxFrame_1 = ErrFluxFrameDict[SetName1]
    err1 = np.zeros((DimY,DimX))
    err1 = ErrMeanFluxFrame_1
    err1 = np.where(HotsArray > 0, 1, err1)
    err1 = np.where(ErrMeanFluxFrame_1 <= 0, 1, err1)
    
    #ZeroErrFrame1 = np.zeros((DimY,DimX))
    #ZeroErrFrame1 = np.where(ErrMeanFluxFrame_1 <= 0, 1, ZeroErrFrame1)

    MeanFluxFrame_2 = FluxFrameDict[SetName2]
    img2 = np.zeros((DimY,DimX))
    img2 =  MeanFluxFrame_2
    img2 = np.where(HotsArray > 0, 0, img2)
    
    
    MosaicX = CoordsDict[SetName1][0]
    MosaicY = CoordsDict[SetName1][1]
    SetCoord_1 = '(' + str(MosaicX) + ',' + str(MosaicY) + ')'
    
    MosaicX = CoordsDict[SetName2][0]
    MosaicY = CoordsDict[SetName2][1]
    SetCoord_2 = '(' + str(MosaicX) + ',' + str(MosaicY) + ')'
    
    #Define the same ROI in the second frame
    ROI2 = [ROI1[0]-dx,ROI1[1]-dx,ROI1[2]-dy,ROI1[3]-dy]
    
    #Checking that x's and y's are within bounds of new frame for the given dx and dy
    if ROI2[0] > -1 and ROI2[1] < DimX + 1 and ROI2[2] > -1 and ROI2[3] < DimY + 1:
    #if ROI1[0] + dx > -1 and ROI1[1] + dx < DimX + 1 and ROI1[2] + dy > -1 and ROI1[3] + dy < DimY + 1:
        
        #initialize ROI_Frames as zeros arrays
        DimX_ROI = ROI1[1] - ROI1[0]
        DimY_ROI = ROI1[3] - ROI1[2]
        ROI_Frame1 = np.zeros((DimY_ROI,DimX_ROI))
        ROI_Frame2 = np.zeros((DimY_ROI,DimX_ROI))
        ROI_ErrFrame1 = np.zeros((DimY_ROI,DimX_ROI))

        ROI_HotsArray1 = np.zeros((DimY_ROI,DimX_ROI))
        ROI_HotsArray2 = np.zeros((DimY_ROI,DimX_ROI))
        #ROI_ZeroError1 = np.zeros((DimY_ROI,DimX_ROI))
        
        ROI_Frame1 = img1[ROI1[2]:ROI1[3],ROI1[0]:ROI1[1]]
        ROI_Frame2 = img2[ROI2[2]:ROI2[3],ROI2[0]:ROI2[1]]
        ROI_ErrFrame1 = err1[ROI1[2]:ROI1[3],ROI1[0]:ROI1[1]]

        ROI_HotsArray1 = HotsArray[ROI1[2]:ROI1[3],ROI1[0]:ROI1[1]]
        ROI_HotsArray2 = HotsArray[ROI2[2]:ROI2[3],ROI2[0]:ROI2[1]]
        #ROI_ZeroError1 = ZeroErrFrame1[ROI1[2]:ROI1[3],ROI1[0]:ROI1[1]]
        
        Combo_ROI_HotsArray = ROI_HotsArray1 + ROI_HotsArray2 #+ ROI_ZeroError1  added the pixels with 0 err
        
        Combo_ROI_HotsArray = np.where(Combo_ROI_HotsArray > 0, 1, Combo_ROI_HotsArray) #and the pixels with zero error
        
        NumberPixNotHot = (DimX_ROI*DimY_ROI) - np.sum(Combo_ROI_HotsArray) #and the pixels with zero error
        
        #Find the error weighted mean per pixel absolute residual (using errors from frame 1)
        ResidualsFrame = (ROI_Frame1 - ROI_Frame2)
        WeightedResidualsFrame = np.absolute(ROI_Frame1 - ROI_Frame2)/np.absolute(ROI_ErrFrame1)
        WeightedResidualsFrame = np.where(Combo_ROI_HotsArray > 0, 0, WeightedResidualsFrame)
        SumWeightedResiduals = np.sum(WeightedResidualsFrame)
        mean_per_pixel_weighted_residuals = SumWeightedResiduals/NumberPixNotHot
        
        if plot1 != False:
            
            if SavePlots == True:
                
                SMALL_SIZE = 15
                MEDIUM_SIZE = 15
                BIGGER_SIZE = 20

                plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
                plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
                plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
                plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
                plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
                plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
                plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
                
            if SavePlots == False:
                matplotlib.rcParams.update(matplotlib.rcParamsDefault)
            
            xROI1, yROI1 = ROItoPoints(ROI1)
            xROI2, yROI2 = ROItoPoints(ROI2)
            
            f, ax = plt.subplots(2, 1)
            im1 = ax[0].imshow(img1, cmap='turbo', vmin = 0, vmax = 5)
            divider = make_axes_locatable(ax[0])
            cax1 = divider.append_axes("right", size="5%", pad=0.15)
            cbar1 = plt.colorbar(im1, cax=cax1)
            cbar1.formatter.set_powerlimits((0, 0))
            ax[0].fill(xROI1, yROI1, facecolor='none', edgecolor='magenta', linewidth=2, label = "ROI1")
            ax[0].set_xlabel('Pixel x')
            ax[0].set_ylabel('Pixel y')
            ax[0].set_title(r"Frame 1 " + SetCoord_1 + " with ROI [$e^{-}/s$]"
                           "\n" r"490.5 eV", loc = 'center')

            im2 = ax[1].imshow(img2, cmap='turbo', vmin = 0, vmax = 5)
            divider = make_axes_locatable(ax[1])
            cax2 = divider.append_axes("right", size="5%", pad=0.15)
            cbar2 = plt.colorbar(im2, cax=cax2)
            cbar2.formatter.set_powerlimits((0, 0))
            ax[1].fill(xROI2, yROI2, facecolor='none', edgecolor='magenta', linewidth=2, label = "ROI2")
            ax[1].set_xlabel('Pixel x')
            ax[1].set_ylabel('Pixel y')
            ax[1].set_title(r"Frame 2 " + SetCoord_2 + " with ROI [$e^{-}/s$]"
                           "\n" r"490.5 eV", loc = 'center')
            
            plt.subplots_adjust(hspace = 0.35)
            figure = plt.gcf()
            
            if SavePlots == False:
                figure.set_size_inches(10, 10)
                plt.show()
            
            if SavePlots == True:
                figure.set_size_inches(13, 13)
                SaveString = "FullFramesWithROI_" + SetName1 + "_" + SetName2 + ".png"
                plt.savefig(SaveString, dpi=300,bbox_inches='tight')
                plt.show()
            
        if plot2 != False:
            
            if SavePlots == True:
                
                SMALL_SIZE = 15
                MEDIUM_SIZE = 15
                BIGGER_SIZE = 20

                plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
                plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
                plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
                plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
                plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
                plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
                plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
                
            if SavePlots == False:
                matplotlib.rcParams.update(matplotlib.rcParamsDefault)
            
            f, ax = plt.subplots(2, 1)
            im1 = ax[0].imshow(ROI_Frame1, cmap='turbo', vmin = 0, vmax = 5)
            divider = make_axes_locatable(ax[0])
            cax1 = divider.append_axes("right", size="5%", pad=0.15)
            cbar1 = plt.colorbar(im1, cax=cax1)
            cbar1.formatter.set_powerlimits((0, 0))
            ax[0].set_xlabel('Pixel x')
            ax[0].set_ylabel('Pixel y')
            ax[0].set_title("ROI from Frame 1 " + SetCoord_1 + " [$e^{-}/s$]490.5 eV")

            im2 = ax[1].imshow(ROI_Frame2, cmap='turbo', vmin = 0, vmax = 5)
            divider = make_axes_locatable(ax[1])
            cax2 = divider.append_axes("right", size="5%", pad=0.15)
            cbar2 = plt.colorbar(im2, cax=cax2)
            cbar2.formatter.set_powerlimits((0, 0))
            ax[1].set_xlabel('Pixel x')
            ax[1].set_ylabel('Pixel y')
            ax[1].set_title("ROI from Frame 2 " + SetCoord_2 + " [$e^{-}/s$]")
            
            plt.subplots_adjust(hspace = 0.35)
            figure = plt.gcf()
            #figure.set_size_inches(10, 6)
            #plt.show()
            
            if SavePlots == False:
                figure.set_size_inches(10, 6)
                plt.show()
            
            if SavePlots == True:
                figure.set_size_inches(13, 7)
                SaveString = "JustROI_" + SetName1 + "_" + SetName2 + ".png"
                plt.savefig(SaveString, dpi=300,bbox_inches='tight')
                plt.show()
            
        if plotResiduals != False:
            
            Vmax = 1000
            
            if SavePlots == True:
                
                SMALL_SIZE = 15
                MEDIUM_SIZE = 15
                BIGGER_SIZE = 20

                plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
                plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
                plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
                plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
                plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
                plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
                plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
                
            if SavePlots == False:
                matplotlib.rcParams.update(matplotlib.rcParamsDefault)
            
            f, ax = plt.subplots(2, 1)
            im1 = ax[0].imshow(WeightedResidualsFrame, cmap='turbo', vmin = 0, vmax = Vmax)
            #im1 = ax[0].imshow(SquaredResiduals, cmap='coolwarm', vmin = 0, vmax = Vmax)
            divider = make_axes_locatable(ax[0])
            cax1 = divider.append_axes("right", size="5%", pad=0.15)
            cbar1 = plt.colorbar(im1, cax=cax1)
            cbar1.formatter.set_powerlimits((0, 0))
            ax[0].set_xlabel('Pixel x')
            ax[0].set_ylabel('Pixel y')
            ax[0].set_title("ROI Error Weighted Residuals " + SetCoord_1 +" and " + SetCoord_2 + " [$e^{-}/s$]")

            #im2 = ax[1].imshow(ResidualsFrame, cmap='turbo', vmin = -300, vmax = 300) #was 150
            #im2 = ax[1].imshow(ResidualsFrame, cmap='coolwarm', vmin = -300, vmax = 300) #was 150
            im2 = ax[1].imshow(ResidualsFrame, cmap='bwr', vmin = -300, vmax = 300) #was 150
            divider = make_axes_locatable(ax[1])
            cax2 = divider.append_axes("right", size="5%", pad=0.15)
            cbar2 = plt.colorbar(im2, cax=cax2)
            cbar2.formatter.set_powerlimits((0, 0))
            ax[1].set_xlabel('Pixel x')
            ax[1].set_ylabel('Pixel y')
            ax[1].set_title("ROI Residuals " + SetCoord_1 +" and " + SetCoord_2 + " [$e^{-}/s$]")
            
            plt.subplots_adjust(hspace = 0.35)
            figure = plt.gcf()
            #figure.set_size_inches(10, 6)
            #plt.show()
            
            if SavePlots == False:
                figure.set_size_inches(10, 6)
                plt.show()
            
            if SavePlots == True:
                figure.set_size_inches(13, 7)
                SaveString = "Residuals_" + SetName1 + "_" + SetName2 + ".png"
                plt.savefig(SaveString, dpi=300,bbox_inches='tight')
                plt.show()
            
        
        
        return mean_per_pixel_weighted_residuals, WeightedResidualsFrame
    
    else: 
        #Need to make it return something to indicate that the dx's and dy's are incompatable with the chosen ROI1 being in
        #both frames
        print("Invalid combination of dx, dy, and ROI1.")
        
        return False, False

In [ ]:
def PlotWeightedQualityFunction(SetName1,SetName2,ROI1,HotsArray,dx,dy,xInches,yInches,ResidualScale,SavePlots=False):
    #Returns the mean per pixel squared residual in the ROI given a dx and dy and the ROI in the first image set
    
    dx = round(dx)
    dy = round(dy)
    
    #Making the two images given the set names and zeroing out the hotpixels
    MeanFluxFrame_1 = FluxFrameDict[SetName1]
    img1 = np.zeros((DimY,DimX))
    img1 =  MeanFluxFrame_1
    img1 = np.where(HotsArray > 0, 0, img1)
    
    ErrMeanFluxFrame_1 = ErrFluxFrameDict[SetName1]
    err1 = np.zeros((DimY,DimX))
    err1 = ErrMeanFluxFrame_1
    err1 = np.where(HotsArray > 0, 1, err1)
    err1 = np.where(ErrMeanFluxFrame_1 <= 0, 1, err1)

    MeanFluxFrame_2 = FluxFrameDict[SetName2]
    img2 = np.zeros((DimY,DimX))
    img2 =  MeanFluxFrame_2
    img2 = np.where(HotsArray > 0, 0, img2)
    
    ErrMeanFluxFrame_2 = ErrFluxFrameDict[SetName2]
    err2 = np.zeros((DimY,DimX))
    err2 = ErrMeanFluxFrame_2
    err2 = np.where(HotsArray > 0, 1, err2)
    err2 = np.where(ErrMeanFluxFrame_2 <= 0, 1, err2)
    
    MosaicX = CoordsDict[SetName1][0]
    MosaicY = CoordsDict[SetName1][1]
    SetCoord_1 = '(' + str(MosaicX) + ',' + str(MosaicY) + ')'
    
    MosaicX = CoordsDict[SetName2][0]
    MosaicY = CoordsDict[SetName2][1]
    SetCoord_2 = '(' + str(MosaicX) + ',' + str(MosaicY) + ')'
    
    #Define the same ROI in the second frame
    ROI2 = [ROI1[0]-dx,ROI1[1]-dx,ROI1[2]-dy,ROI1[3]-dy]
    
    #Checking that x's and y's are within bounds of new frame for the given dx and dy
    if ROI2[0] > -1 and ROI2[1] < DimX + 1 and ROI2[2] > -1 and ROI2[3] < DimY + 1:
    #if ROI1[0] + dx > -1 and ROI1[1] + dx < DimX + 1 and ROI1[2] + dy > -1 and ROI1[3] + dy < DimY + 1:
        
        #initialize ROI_Frames as zeros arrays
        DimX_ROI = ROI1[1] - ROI1[0]
        DimY_ROI = ROI1[3] - ROI1[2]
        ROI_Frame1 = np.zeros((DimY_ROI,DimX_ROI))
        ROI_Frame2 = np.zeros((DimY_ROI,DimX_ROI))
        ROI_ErrFrame1 = np.zeros((DimY_ROI,DimX_ROI))
        ROI_ErrFrame2 = np.zeros((DimY_ROI,DimX_ROI))
        ROI_HotsArray1 = np.zeros((DimY_ROI,DimX_ROI))
        ROI_HotsArray2 = np.zeros((DimY_ROI,DimX_ROI))
        
        ROI_Frame1 = img1[ROI1[2]:ROI1[3],ROI1[0]:ROI1[1]]
        ROI_Frame2 = img2[ROI2[2]:ROI2[3],ROI2[0]:ROI2[1]]
        ROI_ErrFrame1 = err1[ROI1[2]:ROI1[3],ROI1[0]:ROI1[1]]
        ROI_ErrFrame1 = err2[ROI2[2]:ROI2[3],ROI2[0]:ROI2[1]]
        ROI_HotsArray1 = HotsArray[ROI1[2]:ROI1[3],ROI1[0]:ROI1[1]]
        ROI_HotsArray2 = HotsArray[ROI2[2]:ROI2[3],ROI2[0]:ROI2[1]]
        
        Combo_ROI_HotsArray = ROI_HotsArray1 + ROI_HotsArray2
        
        Combo_ROI_HotsArray = np.where(Combo_ROI_HotsArray > 0, 1, Combo_ROI_HotsArray)
        
        NumberPixNotHot = (DimX_ROI*DimY_ROI) - np.sum(Combo_ROI_HotsArray)
        
        #Find the error weighted mean per pixel absolute residual (using errors from frame 1)
        ResidualsFrame = (ROI_Frame1 - ROI_Frame2)
        WeightedResidualsFrame = np.absolute(ROI_Frame1 - ROI_Frame2)/np.absolute(ROI_ErrFrame1)
        WeightedResidualsFrame = np.where(Combo_ROI_HotsArray > 0, 0, WeightedResidualsFrame)
        SumWeightedResiduals = np.sum(WeightedResidualsFrame)
        mean_per_pixel_weighted_residuals = SumWeightedResiduals/NumberPixNotHot
            
        if SavePlots == True:

            SMALL_SIZE = 15
            MEDIUM_SIZE = 15
            BIGGER_SIZE = 20

            plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
            plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
            plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
            plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
            plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
            plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
            plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

        if SavePlots == False:
            matplotlib.rcParams.update(matplotlib.rcParamsDefault)

        xROI1, yROI1 = ROItoPoints(ROI1)
        xROI2, yROI2 = ROItoPoints(ROI2)

        f, ax = plt.subplots(2, 1)
        im1 = ax[0].imshow(img1, cmap='turbo', vmin = 0, vmax = 5)
        divider = make_axes_locatable(ax[0])
        cax1 = divider.append_axes("right", size="5%", pad=0.15)
        cbar1 = plt.colorbar(im1, cax=cax1)
        cbar1.formatter.set_powerlimits((0, 0))
        ax[0].fill(xROI1, yROI1, facecolor='none', edgecolor='magenta', linewidth=2, label = "ROI1")
        ax[0].set_xlabel('Pixel x')
        ax[0].set_ylabel('Pixel y')
        ax[0].set_title(r"Frame 1 " + SetCoord_1 + " with ROI [$e^{-}/s$]"
                       "\n" r"490.5 eV", loc = 'center')

        im2 = ax[1].imshow(img2, cmap='turbo', vmin = 0, vmax = 5)
        divider = make_axes_locatable(ax[1])
        cax2 = divider.append_axes("right", size="5%", pad=0.15)
        cbar2 = plt.colorbar(im2, cax=cax2)
        cbar2.formatter.set_powerlimits((0, 0))
        ax[1].fill(xROI2, yROI2, facecolor='none', edgecolor='magenta', linewidth=2, label = "ROI2")
        ax[1].set_xlabel('Pixel x')
        ax[1].set_ylabel('Pixel y')
        ax[1].set_title(r"Frame 2 " + SetCoord_2 + " with ROI [$e^{-}/s$]"
                       "\n" r"490.5 eV", loc = 'center')

        plt.subplots_adjust(hspace = 0.35)
        figure = plt.gcf()

        if SavePlots == False:
            figure.set_size_inches(10, 10)
            plt.show()

        if SavePlots == True:
            figure.set_size_inches(13, 13)
            SaveString = "FullFramesWithROI_" + SetName1 + "_and_" + SetName2 + ".png"
            plt.savefig(SaveString, dpi=300,bbox_inches='tight')
            plt.show()
            
        if SavePlots == True:

            SMALL_SIZE = 15
            MEDIUM_SIZE = 15
            BIGGER_SIZE = 20

            plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
            plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
            plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
            plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
            plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
            plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
            plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

        if SavePlots == False:
            matplotlib.rcParams.update(matplotlib.rcParamsDefault)

        f, ax = plt.subplots(2, 1)
        im1 = ax[0].imshow(ROI_Frame1, cmap='turbo', vmin = 0, vmax = 5)
        divider = make_axes_locatable(ax[0])
        cax1 = divider.append_axes("right", size="5%", pad=0.15)
        cbar1 = plt.colorbar(im1, cax=cax1)
        cbar1.formatter.set_powerlimits((0, 0))
        ax[0].set_xlabel('Pixel x')
        ax[0].set_ylabel('Pixel y')
        ax[0].set_title("ROI from Frame 1 " + SetCoord_1 + " [$e^{-}/s$]", y=1.05)

        im2 = ax[1].imshow(ROI_Frame2, cmap='turbo', vmin = 0, vmax = 5)
        divider = make_axes_locatable(ax[1])
        cax2 = divider.append_axes("right", size="5%", pad=0.15)
        cbar2 = plt.colorbar(im2, cax=cax2)
        cbar2.formatter.set_powerlimits((0, 0))
        ax[1].set_xlabel('Pixel x')
        ax[1].set_ylabel('Pixel y')
        ax[1].set_title("ROI from Frame 2 " + SetCoord_2 + " [$e^{-}/s$]", y=1.05)

        plt.subplots_adjust(hspace = 0.35)
        figure = plt.gcf()
        #figure.set_size_inches(10, 6)
        #plt.show()

        if SavePlots == False:
            #figure.set_size_inches(10, 6)
            figure.set_size_inches(xInches, yInches)
            plt.show()

        if SavePlots == True:
            figure.set_size_inches(xInches, yInches)
            SaveString = "JustROI_" + SetName1 + "_and_" + SetName2 + ".png"
            plt.savefig(SaveString, dpi=300,bbox_inches='tight')
            plt.show()
            
        Vmax = 1000

        if SavePlots == True:

            SMALL_SIZE = 15
            MEDIUM_SIZE = 15
            BIGGER_SIZE = 20

            plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
            plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
            plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
            plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
            plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
            plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
            plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

        if SavePlots == False:
            matplotlib.rcParams.update(matplotlib.rcParamsDefault)

        f, ax = plt.subplots(2, 1)
        im1 = ax[0].imshow(WeightedResidualsFrame, cmap='turbo', vmin = 0, vmax = Vmax)
        #im1 = ax[0].imshow(SquaredResiduals, cmap='coolwarm', vmin = 0, vmax = Vmax)
        divider = make_axes_locatable(ax[0])
        cax1 = divider.append_axes("right", size="5%", pad=0.15)
        cbar1 = plt.colorbar(im1, cax=cax1)
        cbar1.formatter.set_powerlimits((0, 0))
        ax[0].set_xlabel('Pixel x')
        ax[0].set_ylabel('Pixel y')
        ax[0].set_title("ROI Error Weighted Residuals " + SetCoord_1 +" and " + SetCoord_2 + " [$e^{-}/s$]", y=1.05)

        #im2 = ax[1].imshow(ResidualsFrame, cmap='turbo', vmin = -300, vmax = 300) #was 150
        #im2 = ax[1].imshow(ResidualsFrame, cmap='coolwarm', vmin = -300, vmax = 300) #was 150
        im2 = ax[1].imshow(ResidualsFrame, cmap='bwr', vmin = -1*ResidualScale, vmax = ResidualScale) #was 150
        divider = make_axes_locatable(ax[1])
        cax2 = divider.append_axes("right", size="5%", pad=0.15)
        cbar2 = plt.colorbar(im2, cax=cax2)
        cbar2.formatter.set_powerlimits((0, 0))
        ax[1].set_xlabel('Pixel x')
        ax[1].set_ylabel('Pixel y')
        ax[1].set_title("ROI Residuals " + SetCoord_1 +" and " + SetCoord_2 + " [$e^{-}/s$]", y=1.05)

        plt.subplots_adjust(hspace = 0.35)
        figure = plt.gcf()
        #figure.set_size_inches(10, 6)
        #plt.show()

        if SavePlots == False:
            #figure.set_size_inches(10, 6)
            figure.set_size_inches(xInches, yInches)
            plt.show()

        if SavePlots == True:
            figure.set_size_inches(xInches, yInches)
            SaveString = "Residuals_" + SetName1 + "_and_" + SetName2 + ".png"
            plt.savefig(SaveString, dpi=300,bbox_inches='tight')
            plt.show()
    
    else: 
        #Need to make it return something to indicate that the dx's and dy's are incompatable with the chosen ROI1 being in
        #both frames
        print("Invalid combination of dx, dy, and ROI1.")

In [ ]:
def compute_shift_gd3(SetName1,SetName2,ROI1,HotsArray,dx0,dy0,scooch_dimension,verbose=True):
    
    #Keeping variable names from gd2 even though they now mean different things from what they say.
    
    MeanFluxFrame_1 = FluxFrameDict[SetName1]
    img1 = np.zeros((DimY,DimX))
    img1 =  MeanFluxFrame_1
    img1 = np.where(HotsArray > 0, 0, img1)
    
    MeanFluxFrame_2 = FluxFrameDict[SetName2]
    img2 = np.zeros((DimY,DimX))
    img2 =  MeanFluxFrame_2
    img2 = np.where(HotsArray > 0, 0, img2)
    
    scooch = int((scooch_dimension - 1)/2)
    #scooch = 2
    
    dx = round(dx0)
    dy = round(dy0)
    mean_squared_residual, SquaredResidualsFrame = WeightedQualityFunction(SetName1,SetName2,ROI1,HotsArray,dx,dy)
    #Keeping variable names from gd2 even though they now mean different things from what they say.
    #mean_squared_residual, SquaredResidualsFrame = mean_per_pixel_weighted_residuals, WeightedResidualsFrame in truth
    if mean_squared_residual != False:

        new_translation = (dx,dy)
        new_translations_list = [new_translation]

        mean_residuals = [mean_squared_residual]
        #NumOverlapPixelsList = [NumOverlapPixels]

        prev_min_mean_squared_residual = mean_squared_residual

        min_mean_squared_residual = 10**15 #initalizing a value for abs_residual_sum for the for loop, will not be saved

        while min_mean_squared_residual != prev_min_mean_squared_residual:
        #while min_abs_residual_sum/NumOverlapPixels != prev_min_abs_residual_sum:
            #Will keep going until the previous abs
            for x_increment in range(-1*scooch,scooch + 1):
                for y_increment in range(-1*scooch,scooch + 1):

                    new_dx = dx + x_increment
                    new_dy = dy + y_increment

                    new_translation = (new_dx,new_dy)

                    if new_translation not in new_translations_list: 
                        # if tuple (new_dx,new_dy) not in the associated list of translation tuples
                        mean_squared_residual, SquaredResidualsFrame = WeightedQualityFunction(SetName1,SetName2,ROI1,HotsArray,new_dx,new_dy)
                        if mean_squared_residual != False:
                            mean_residuals.append(mean_squared_residual)
                            #NumOverlapPixelsList.append(NumOverlapPixels)
                            #new_translation = (new_dx,new_dy)
                            new_translations_list.append(new_translation)

                            current_min_mean_squared_residual = min(mean_residuals)

            #Find index of min_abs_residual_sum in list of (abs) residual sums
            min_index = mean_residuals.index(current_min_mean_squared_residual)

            #define a new dx = dx0 and dy = dy0 (of minimum residual sum) here to go through the while loop again
            the_translation_of_min = new_translations_list[min_index]
            dx = the_translation_of_min[0]
            dy = the_translation_of_min[1]

            if verbose == True:
                R_SquaredString  = "{:.4e}".format(current_min_mean_squared_residual)
                print("Best so far: dx = " + str(round(dx)) + " dy = " + str(round(dy)) + " Mean(abs(R)/err) = " + R_SquaredString)

            min_mean_squared_residual = current_min_mean_squared_residual

            if min_mean_squared_residual == prev_min_mean_squared_residual:

                break
            else:
                prev_min_mean_squared_residual = min_mean_squared_residual
                min_mean_squared_residual = 10**15
        
        #Renaming varaibles to correct names before returning them
        mean_weighted_residuals = mean_residuals
        min_mean_weighted_residual = min_mean_squared_residual
        return dx, dy, new_translations_list, mean_weighted_residuals, min_mean_weighted_residual
    
    else:
        return False, False, False, False, False

In [ ]:
def compute_shift_gd_it2(KnownFrame,UnknownFrame,ROI1,HotsArray,dx0,dy0,verbose=False):
    #Code to itterate compute_shift_gd3 with different starting points
    
    additions = [0,100,-100,150,-150,200,-200]
    
    dx0_list = []
    dy0_list = []
    dx_list = []
    dy_list = []
    residual_list = []

    for dx_addition in tqdm(additions):
        for dy_addition in additions:
            dx1 = dx0 + dx_addition
            dy1 = dy0 + dy_addition

            #print("Gradient Descent Starting Point: dx0 = " + str(round(dx0)) + " dy0 = " + str(round(dy0)))

            dx, dy, new_translations_list, mean_weighted_residuals, min_mean_weighted_residual = compute_shift_gd3(KnownFrame,UnknownFrame,ROI1,HotsArray,dx1,dy1,3,verbose=False)
            
            #print("")

            if dx != False:
                dx_list.append(dx)
                dy_list.append(dy)
                dx0_list.append(dx1)
                dy0_list.append(dy1)

                residual_list.append(min_mean_weighted_residual)
                
                if verbose == True:
                    print("")
                    print("---------------------------------------------------------------------------------------")
                    print("Gradient Descent Starting Point: dx0 = " + str(round(dx1)) + " dy0 = " + str(round(dy1)))
                    R_String  = "{:.4e}".format(min_mean_weighted_residual)
                    #print("Best so far: dx = " + str(round(dx)) + " dy = " + str(round(dy)) + " Mean(abs(R)/err) = " + R_String)
                    print("Best so far: dx = " + str(round(dx)) + " dy = " + str(round(dy)))
                    print("Mean Per Pixel Error Weighted Residual in ROI: " + R_String)
                    print("---------------------------------------------------------------------------------------")
                    print("")
                
    min_mean_weighted_residual = min(residual_list)
    min_index = residual_list.index(min_mean_weighted_residual)
    
    dx = dx_list[min_index]
    dy = dy_list[min_index]
    
    return dx, dy, min_mean_weighted_residual

In [ ]:
#Begin the new Mosaic by aligning X0_Y0 and X0_YM1 gradient descent (using dx0 and dy0 from the previous attempt)
#Begin by using HelpPickROI function to pick a hopefully valid overlaping ROI
KnownFrame = 'X0_Y0'
UnknownFrame = 'X0_YM1'
dx, dy = Recall_dx_dy(KnownFrame,UnknownFrame,TranslateX_Dict_GD_Test2,TranslateY_Dict_GD_Test2)

ROI1=[250,1750,710,1050]

HelpPickROI(KnownFrame,UnknownFrame,ROI1,HotsArray,dx,dy)

In [ ]:
#Getting the dx0 and dy0 from a previous attempt at gradient descent 
dx0, dy0 = Recall_dx_dy(KnownFrame,UnknownFrame,TranslateX_Dict_GD_Test2,TranslateY_Dict_GD_Test2)

dx, dy, min_mean_weighted_residual = compute_shift_gd_it2(KnownFrame,UnknownFrame,ROI1,HotsArray,dx0,dy0,verbose=True)
R_String  = "{:.4e}".format(min_mean_weighted_residual)
print("")
print("---------------------------------------------------------------------------------------")
print("---------------------------------------------------------------------------------------")
print("Gradient Descent Starting Point: dx0 = " + str(round(dx0)) + " dy0 = " + str(round(dy0)))
print("Best Gradient Descent Result: dx = " + str(round(dx)) + " dy = " + str(round(dy)))
print("Mean Per Pixel Error Weighted Residual in ROI: " + R_String)
print("---------------------------------------------------------------------------------------")
print("---------------------------------------------------------------------------------------")
print("")

mean_per_pixel_weighted_residuals, WeightedResidualsFrame = WeightedQualityFunction(KnownFrame,UnknownFrame,ROI1,HotsArray,dx,dy,
                                                                          plot1=True,plot2=True,
                                                                          plotResiduals=True)

In [ ]:
#Updating and saving the Translation Dictionaries with the new Translations.
TranslateX0 = TranslateX_Dict_GD4[KnownFrame]
TranslateY0 = TranslateY_Dict_GD4[KnownFrame]

TranslateX_Dict_GD4.update({UnknownFrame:int(TranslateX0+dx)})
TranslateY_Dict_GD4.update({UnknownFrame:int(TranslateY0+dy)})

with open('TranslateX_Dict_GD4.pkl', 'wb') as f:
    pickle.dump(TranslateX_Dict_GD4, f)
        
with open('TranslateY_Dict_GD4.pkl', 'wb') as f:
    pickle.dump(TranslateY_Dict_GD4, f)

In [ ]:
#Loading in the new updated and saved Translation Dictionaries 
with open('TranslateX_Dict_GD4.pkl', 'rb') as f:
    TranslateX_Dict_GD4 = pickle.load(f)
        
with open('TranslateY_Dict_GD4.pkl', 'rb') as f:
    TranslateY_Dict_GD4 = pickle.load(f)

KnownFrame = 'X0_Y0'
UnknownFrame = 'X0_YM1'

#Ploting and maybe saving QualityFunction function
dx0, dy0 = Recall_dx_dy(KnownFrame,UnknownFrame,TranslateX_Dict_GD4,TranslateY_Dict_GD4)

xInches = 13
yInches = 7
ResidualScale = 300

ROI1=[250,1750,710,1050]

PlotWeightedQualityFunction(KnownFrame,UnknownFrame,ROI1,HotsArray,dx0,dy0,xInches,yInches,ResidualScale,SavePlots=False)

In [ ]:
KnownFrame = 'X0_Y0'
UnknownFrame = 'XM2_Y0'

#Begin by using HelpPickROI function to pick a hopefully valid overlaping ROI
dx0, dy0 = Recall_dx_dy(KnownFrame,UnknownFrame,TranslateX_Dict_GD_Test2,TranslateY_Dict_GD_Test2)

ROI=[1450,1870,70,980]

HelpPickROI(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0)

In [ ]:
#Performing Iterated Gradient Descent
dx, dy, min_mean_weighted_residual = compute_shift_gd_it2(KnownFrame,UnknownFrame,ROI1,HotsArray,dx0,dy0,verbose=True)
R_String  = "{:.4e}".format(min_mean_weighted_residual)
print("")
print("---------------------------------------------------------------------------------------")
print("---------------------------------------------------------------------------------------")
print("Gradient Descent Starting Point: dx0 = " + str(round(dx0)) + " dy0 = " + str(round(dy0)))
print("Best Gradient Descent Result: dx = " + str(round(dx)) + " dy = " + str(round(dy)))
print("Mean Per Pixel Error Weighted Residual in ROI: " + R_String)
print("---------------------------------------------------------------------------------------")
print("---------------------------------------------------------------------------------------")
print("")

In [ ]:
#Updating and saving the Translation Dictionaries with the new Translations.
TranslateX0 = TranslateX_Dict_GD4[KnownFrame]
TranslateY0 = TranslateY_Dict_GD4[KnownFrame]

TranslateX_Dict_GD4.update({UnknownFrame:int(TranslateX0+dx)})
TranslateY_Dict_GD4.update({UnknownFrame:int(TranslateY0+dy)})

with open('TranslateX_Dict_GD4.pkl', 'wb') as f:
    pickle.dump(TranslateX_Dict_GD4, f)
        
with open('TranslateY_Dict_GD4.pkl', 'wb') as f:
    pickle.dump(TranslateY_Dict_GD4, f)

In [ ]:
#Loading in the new updated and saved Translation Dictionaries 
with open('TranslateX_Dict_GD4.pkl', 'rb') as f:
    TranslateX_Dict_GD4 = pickle.load(f)
        
with open('TranslateY_Dict_GD4.pkl', 'rb') as f:
    TranslateY_Dict_GD4 = pickle.load(f)


#Ploting and maybe saving QualityFunction function
dx0, dy0 = Recall_dx_dy(KnownFrame,UnknownFrame,TranslateX_Dict_GD4,TranslateY_Dict_GD4)

xInches = 7
yInches = 13
ResidualScale = 300

PlotWeightedQualityFunction(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0,xInches,yInches,ResidualScale,SavePlots=False)

In [ ]:
KnownFrame = 'X0_Y0'
UnknownFrame = 'X0_YP1'

#Begin by using HelpPickROI function to pick a hopefully valid overlaping ROI
dx0, dy0 = Recall_dx_dy(KnownFrame,UnknownFrame,TranslateX_Dict_GD_Test2,TranslateY_Dict_GD_Test2)

ROI=[50,1750,50,380]

HelpPickROI(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0)

In [ ]:
#Performing Iterated Gradient Descent
dx, dy, min_mean_weighted_residual = compute_shift_gd_it2(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0,verbose=True)
R_String  = "{:.4e}".format(min_mean_weighted_residual)
print("")
print("---------------------------------------------------------------------------------------")
print("---------------------------------------------------------------------------------------")
print("Gradient Descent Starting Point: dx0 = " + str(round(dx0)) + " dy0 = " + str(round(dy0)))
print("Best Gradient Descent Result: dx = " + str(round(dx)) + " dy = " + str(round(dy)))
print("Mean Per Pixel Error Weighted Residual in ROI: " + R_String)
print("---------------------------------------------------------------------------------------")
print("---------------------------------------------------------------------------------------")
print("")

In [ ]:
#Updating and saving the Translation Dictionaries with the new Translations.
TranslateX0 = TranslateX_Dict_GD4[KnownFrame]
TranslateY0 = TranslateY_Dict_GD4[KnownFrame]

TranslateX_Dict_GD4.update({UnknownFrame:int(TranslateX0+dx)})
TranslateY_Dict_GD4.update({UnknownFrame:int(TranslateY0+dy)})

with open('TranslateX_Dict_GD4.pkl', 'wb') as f:
    pickle.dump(TranslateX_Dict_GD4, f)
        
with open('TranslateY_Dict_GD4.pkl', 'wb') as f:
    pickle.dump(TranslateY_Dict_GD4, f)

In [ ]:
#Loading in the new updated and saved Translation Dictionaries 
with open('TranslateX_Dict_GD4.pkl', 'rb') as f:
    TranslateX_Dict_GD4 = pickle.load(f)
        
with open('TranslateY_Dict_GD4.pkl', 'rb') as f:
    TranslateY_Dict_GD4 = pickle.load(f)


#Ploting and maybe saving QualityFunction function
dx0, dy0 = Recall_dx_dy(KnownFrame,UnknownFrame,TranslateX_Dict_GD4,TranslateY_Dict_GD4)

xInches = 13
yInches = 7
ResidualScale = 300

PlotWeightedQualityFunction(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0,xInches,yInches,ResidualScale,SavePlots=False)

In [ ]:
KnownFrame = 'X0_Y0'
UnknownFrame = 'XP2_Y0'

#Begin by using HelpPickROI function to pick a hopefully valid overlaping ROI
dx0, dy0 = Recall_dx_dy(KnownFrame,UnknownFrame,TranslateX_Dict_GD_Test2,TranslateY_Dict_GD_Test2)

ROI=[50,650,120,1040]

HelpPickROI(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0)

In [ ]:
#Performing Iterated Gradient Descent
dx, dy, min_mean_weighted_residual = compute_shift_gd_it2(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0,verbose=True)
R_String  = "{:.4e}".format(min_mean_weighted_residual)
print("")
print("---------------------------------------------------------------------------------------")
print("---------------------------------------------------------------------------------------")
print("Gradient Descent Starting Point: dx0 = " + str(round(dx0)) + " dy0 = " + str(round(dy0)))
print("Best Gradient Descent Result: dx = " + str(round(dx)) + " dy = " + str(round(dy)))
print("Mean Per Pixel Error Weighted Residual in ROI: " + R_String)
print("---------------------------------------------------------------------------------------")
print("---------------------------------------------------------------------------------------")
print("")

In [ ]:
#Updating and saving the Translation Dictionaries with the new Translations.
TranslateX0 = TranslateX_Dict_GD4[KnownFrame]
TranslateY0 = TranslateY_Dict_GD4[KnownFrame]

TranslateX_Dict_GD4.update({UnknownFrame:int(TranslateX0+dx)})
TranslateY_Dict_GD4.update({UnknownFrame:int(TranslateY0+dy)})

with open('TranslateX_Dict_GD4.pkl', 'wb') as f:
    pickle.dump(TranslateX_Dict_GD4, f)
        
with open('TranslateY_Dict_GD4.pkl', 'wb') as f:
    pickle.dump(TranslateY_Dict_GD4, f)

In [ ]:
#Loading in the new updated and saved Translation Dictionaries 
with open('TranslateX_Dict_GD4.pkl', 'rb') as f:
    TranslateX_Dict_GD4 = pickle.load(f)
        
with open('TranslateY_Dict_GD4.pkl', 'rb') as f:
    TranslateY_Dict_GD4 = pickle.load(f)


#Ploting and maybe saving QualityFunction function
dx0, dy0 = Recall_dx_dy(KnownFrame,UnknownFrame,TranslateX_Dict_GD4,TranslateY_Dict_GD4)

xInches = 7
yInches = 13
ResidualScale = 300

PlotWeightedQualityFunction(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0,xInches,yInches,ResidualScale,SavePlots=False)

In [ ]:
KnownFrame = 'X0_YP1'
UnknownFrame = 'X0_YP2'

#Begin by using HelpPickROI function to pick a hopefully valid overlaping ROI
dx0, dy0 = Recall_dx_dy(KnownFrame,UnknownFrame,TranslateX_Dict_GD_Test2,TranslateY_Dict_GD_Test2)

ROI=[210,1700,50,350]

HelpPickROI(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0)

In [ ]:
#Performing Iterated Gradient Descent
dx, dy, min_mean_weighted_residual = compute_shift_gd_it2(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0,verbose=True)
R_String  = "{:.4e}".format(min_mean_weighted_residual)
print("")
print("---------------------------------------------------------------------------------------")
print("---------------------------------------------------------------------------------------")
print("Gradient Descent Starting Point: dx0 = " + str(round(dx0)) + " dy0 = " + str(round(dy0)))
print("Best Gradient Descent Result: dx = " + str(round(dx)) + " dy = " + str(round(dy)))
print("Mean Per Pixel Error Weighted Residual in ROI: " + R_String)
print("---------------------------------------------------------------------------------------")
print("---------------------------------------------------------------------------------------")
print("")

In [ ]:
#Updating and saving the Translation Dictionaries with the new Translations.
TranslateX0 = TranslateX_Dict_GD4[KnownFrame]
TranslateY0 = TranslateY_Dict_GD4[KnownFrame]

TranslateX_Dict_GD4.update({UnknownFrame:int(TranslateX0+dx)})
TranslateY_Dict_GD4.update({UnknownFrame:int(TranslateY0+dy)})

with open('TranslateX_Dict_GD4.pkl', 'wb') as f:
    pickle.dump(TranslateX_Dict_GD4, f)
        
with open('TranslateY_Dict_GD4.pkl', 'wb') as f:
    pickle.dump(TranslateY_Dict_GD4, f)

In [ ]:
#Loading in the new updated and saved Translation Dictionaries 
with open('TranslateX_Dict_GD4.pkl', 'rb') as f:
    TranslateX_Dict_GD4 = pickle.load(f)
        
with open('TranslateY_Dict_GD4.pkl', 'rb') as f:
    TranslateY_Dict_GD4 = pickle.load(f)


#Ploting and maybe saving QualityFunction function
dx0, dy0 = Recall_dx_dy(KnownFrame,UnknownFrame,TranslateX_Dict_GD4,TranslateY_Dict_GD4)

xInches = 13
yInches = 7
ResidualScale = 300

PlotWeightedQualityFunction(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0,xInches,yInches,ResidualScale,SavePlots=False)

In [ ]:
KnownFrame = 'XP2_Y0'
UnknownFrame = 'XP4_Y0'

#Begin by using HelpPickROI function to pick a hopefully valid overlaping ROI
dx0, dy0 = Recall_dx_dy(KnownFrame,UnknownFrame,TranslateX_Dict_GD4,TranslateY_Dict_GD4)

ROI=[75,560,120,1040]

HelpPickROI(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0)

In [ ]:
#Performing Itterated Gradient Descent
dx, dy, min_mean_weighted_residual = compute_shift_gd_it2(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0,verbose=True)
R_String  = "{:.4e}".format(min_mean_weighted_residual)
print("")
print("---------------------------------------------------------------------------------------")
print("---------------------------------------------------------------------------------------")
print("Gradient Descent Starting Point: dx0 = " + str(round(dx0)) + " dy0 = " + str(round(dy0)))
print("Best Gradient Descent Result: dx = " + str(round(dx)) + " dy = " + str(round(dy)))
print("Mean Per Pixel Error Weighted Residual in ROI: " + R_String)
print("---------------------------------------------------------------------------------------")
print("---------------------------------------------------------------------------------------")
print("")

In [ ]:
#Updating and saving the Translation Dictionaries with the new Translations.
TranslateX0 = TranslateX_Dict_GD4[KnownFrame]
TranslateY0 = TranslateY_Dict_GD4[KnownFrame]

TranslateX_Dict_GD4.update({UnknownFrame:int(TranslateX0+dx)})
TranslateY_Dict_GD4.update({UnknownFrame:int(TranslateY0+dy)})

with open('TranslateX_Dict_GD4.pkl', 'wb') as f:
    pickle.dump(TranslateX_Dict_GD4, f)
        
with open('TranslateY_Dict_GD4.pkl', 'wb') as f:
    pickle.dump(TranslateY_Dict_GD4, f)

In [ ]:
#Loading in the new updated and saved Translation Dictionaries 
with open('TranslateX_Dict_GD4.pkl', 'rb') as f:
    TranslateX_Dict_GD4 = pickle.load(f)
        
with open('TranslateY_Dict_GD4.pkl', 'rb') as f:
    TranslateY_Dict_GD4 = pickle.load(f)


#Ploting and maybe saving QualityFunction function
dx0, dy0 = Recall_dx_dy(KnownFrame,UnknownFrame,TranslateX_Dict_GD4,TranslateY_Dict_GD4)

xInches = 7
yInches = 13
ResidualScale = 300

PlotWeightedQualityFunction(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0,xInches,yInches,ResidualScale,SavePlots=False)

In [ ]:
KnownFrame = 'X0_YM1'
UnknownFrame = 'X0_YM2'

#Begin by using HelpPickROI function to pick a hopefully valid overlaping ROI
dx0, dy0 = Recall_dx_dy(KnownFrame,UnknownFrame,TranslateX_Dict_GD_Test2,TranslateY_Dict_GD_Test2)

ROI=[200,1700,750,1080]

HelpPickROI(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0)

In [ ]:
#Performing Itterated Gradient Descent
dx, dy, min_mean_weighted_residual = compute_shift_gd_it2(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0,verbose=True)
R_String  = "{:.4e}".format(min_mean_weighted_residual)
print("")
print("---------------------------------------------------------------------------------------")
print("---------------------------------------------------------------------------------------")
print("Gradient Descent Starting Point: dx0 = " + str(round(dx0)) + " dy0 = " + str(round(dy0)))
print("Best Gradient Descent Result: dx = " + str(round(dx)) + " dy = " + str(round(dy)))
print("Mean Per Pixel Error Weighted Residual in ROI: " + R_String)
print("---------------------------------------------------------------------------------------")
print("---------------------------------------------------------------------------------------")
print("")

In [ ]:
#Updating and saving the Translation Dictionaries with the new Translations.
TranslateX0 = TranslateX_Dict_GD4[KnownFrame]
TranslateY0 = TranslateY_Dict_GD4[KnownFrame]

TranslateX_Dict_GD4.update({UnknownFrame:int(TranslateX0+dx)})
TranslateY_Dict_GD4.update({UnknownFrame:int(TranslateY0+dy)})

with open('TranslateX_Dict_GD4.pkl', 'wb') as f:
    pickle.dump(TranslateX_Dict_GD4, f)
        
with open('TranslateY_Dict_GD4.pkl', 'wb') as f:
    pickle.dump(TranslateY_Dict_GD4, f)

In [ ]:
#Loading in the new updated and saved Translation Dictionaries 
with open('TranslateX_Dict_GD4.pkl', 'rb') as f:
    TranslateX_Dict_GD4 = pickle.load(f)
        
with open('TranslateY_Dict_GD4.pkl', 'rb') as f:
    TranslateY_Dict_GD4 = pickle.load(f)


#Ploting and maybe saving QualityFunction function
dx0, dy0 = Recall_dx_dy(KnownFrame,UnknownFrame,TranslateX_Dict_GD4,TranslateY_Dict_GD4)

xInches = 13
yInches = 7
ResidualScale = 300

PlotWeightedQualityFunction(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0,xInches,yInches,ResidualScale,SavePlots=False)

In [ ]:
KnownFrame = 'X0_YP2'
UnknownFrame = 'X0_YP3'

#Begin by using HelpPickROI function to pick a hopefully valid overlaping ROI
dx0, dy0 = Recall_dx_dy(KnownFrame,UnknownFrame,TranslateX_Dict_GD_Test2,TranslateY_Dict_GD_Test2)

ROI=[210,1700,50,350]

HelpPickROI(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0)

In [ ]:
#Performing Itterated Gradient Descent
dx, dy, min_mean_weighted_residual = compute_shift_gd_it2(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0,verbose=True)
R_String  = "{:.4e}".format(min_mean_weighted_residual)
print("")
print("---------------------------------------------------------------------------------------")
print("---------------------------------------------------------------------------------------")
print("Gradient Descent Starting Point: dx0 = " + str(round(dx0)) + " dy0 = " + str(round(dy0)))
print("Best Gradient Descent Result: dx = " + str(round(dx)) + " dy = " + str(round(dy)))
print("Mean Per Pixel Error Weighted Residual in ROI: " + R_String)
print("---------------------------------------------------------------------------------------")
print("---------------------------------------------------------------------------------------")
print("")

In [ ]:
#Updating and saving the Translation Dictionaries with the new Translations.
TranslateX0 = TranslateX_Dict_GD4[KnownFrame]
TranslateY0 = TranslateY_Dict_GD4[KnownFrame]

TranslateX_Dict_GD4.update({UnknownFrame:int(TranslateX0+dx)})
TranslateY_Dict_GD4.update({UnknownFrame:int(TranslateY0+dy)})

with open('TranslateX_Dict_GD4.pkl', 'wb') as f:
    pickle.dump(TranslateX_Dict_GD4, f)
        
with open('TranslateY_Dict_GD4.pkl', 'wb') as f:
    pickle.dump(TranslateY_Dict_GD4, f)

In [ ]:
#Loading in the new updated and saved Translation Dictionaries 
with open('TranslateX_Dict_GD4.pkl', 'rb') as f:
    TranslateX_Dict_GD4 = pickle.load(f)
        
with open('TranslateY_Dict_GD4.pkl', 'rb') as f:
    TranslateY_Dict_GD4 = pickle.load(f)


#Ploting and maybe saving QualityFunction function
dx0, dy0 = Recall_dx_dy(KnownFrame,UnknownFrame,TranslateX_Dict_GD4,TranslateY_Dict_GD4)

xInches = 13
yInches = 7
ResidualScale = 300

PlotWeightedQualityFunction(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0,xInches,yInches,ResidualScale,SavePlots=False)

In [ ]:
KnownFrame = 'X0_YM2'
UnknownFrame = 'X0_YM3'

#Begin by using HelpPickROI function to pick a hopefully valid overlaping ROI
dx0, dy0 = Recall_dx_dy(KnownFrame,UnknownFrame,TranslateX_Dict_GD_Test2,TranslateY_Dict_GD_Test2)

ROI=[210,1700,700,1080]

HelpPickROI(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0)

In [ ]:
#Performing Iterated Gradient Descent
dx, dy, min_mean_weighted_residual = compute_shift_gd_it2(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0,verbose=True)
R_String  = "{:.4e}".format(min_mean_weighted_residual)
print("")
print("---------------------------------------------------------------------------------------")
print("---------------------------------------------------------------------------------------")
print("------------------------------- " + KnownFrame + " and " + UnknownFrame + " -------------------------------")
print("Gradient Descent Starting Point: dx0 = " + str(round(dx0)) + " dy0 = " + str(round(dy0)))
print("Best Gradient Descent Result: dx = " + str(round(dx)) + " dy = " + str(round(dy)))
print("Mean Per Pixel Error Weighted Residual in ROI: " + R_String)
print("---------------------------------------------------------------------------------------")
print("---------------------------------------------------------------------------------------")
print("")

In [ ]:
#Updating and saving the Translation Dictionaries with the new Translations.
TranslateX0 = TranslateX_Dict_GD4[KnownFrame]
TranslateY0 = TranslateY_Dict_GD4[KnownFrame]

TranslateX_Dict_GD4.update({UnknownFrame:int(TranslateX0+dx)})
TranslateY_Dict_GD4.update({UnknownFrame:int(TranslateY0+dy)})

with open('TranslateX_Dict_GD4.pkl', 'wb') as f:
    pickle.dump(TranslateX_Dict_GD4, f)
        
with open('TranslateY_Dict_GD4.pkl', 'wb') as f:
    pickle.dump(TranslateY_Dict_GD4, f)

In [ ]:
#Loading in the new updated and saved Translation Dictionaries 
with open('TranslateX_Dict_GD4.pkl', 'rb') as f:
    TranslateX_Dict_GD4 = pickle.load(f)
        
with open('TranslateY_Dict_GD4.pkl', 'rb') as f:
    TranslateY_Dict_GD4 = pickle.load(f)


#Ploting and maybe saving QualityFunction function
dx0, dy0 = Recall_dx_dy(KnownFrame,UnknownFrame,TranslateX_Dict_GD4,TranslateY_Dict_GD4)

xInches = 13
yInches = 7
ResidualScale = 300

PlotWeightedQualityFunction(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0,xInches,yInches,ResidualScale,SavePlots=False)

In [ ]:
KnownFrame = 'X0_YP1'
UnknownFrame = 'XM2_YP1'

#Begin by using HelpPickROI function to pick a hopefully valid overlaping ROI
dx0, dy0 = Recall_dx_dy(KnownFrame,UnknownFrame,TranslateX_Dict_GD_Test2,TranslateY_Dict_GD_Test2)

ROI=[1420,1850,100,1030]

HelpPickROI(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0)

In [ ]:
#Performing Iterated Gradient Descent
dx, dy, min_mean_weighted_residual = compute_shift_gd_it2(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0,verbose=True)
R_String  = "{:.4e}".format(min_mean_weighted_residual)
print("")
print("---------------------------------------------------------------------------------------")
print("---------------------------------------------------------------------------------------")
print("------------------------------- " + KnownFrame + " and " + UnknownFrame + " -------------------------------")
print("Gradient Descent Starting Point: dx0 = " + str(round(dx0)) + " dy0 = " + str(round(dy0)))
print("Best Gradient Descent Result: dx = " + str(round(dx)) + " dy = " + str(round(dy)))
print("Mean Per Pixel Error Weighted Residual in ROI: " + R_String)
print("---------------------------------------------------------------------------------------")
print("---------------------------------------------------------------------------------------")
print("")

In [ ]:
#Updating and saving the Translation Dictionaries with the new Translations.
TranslateX0 = TranslateX_Dict_GD4[KnownFrame]
TranslateY0 = TranslateY_Dict_GD4[KnownFrame]

TranslateX_Dict_GD4.update({UnknownFrame:int(TranslateX0+dx)})
TranslateY_Dict_GD4.update({UnknownFrame:int(TranslateY0+dy)})

with open('TranslateX_Dict_GD4.pkl', 'wb') as f:
    pickle.dump(TranslateX_Dict_GD4, f)
        
with open('TranslateY_Dict_GD4.pkl', 'wb') as f:
    pickle.dump(TranslateY_Dict_GD4, f)

In [ ]:
#Loading in the new updated and saved Translation Dictionaries 
with open('TranslateX_Dict_GD4.pkl', 'rb') as f:
    TranslateX_Dict_GD4 = pickle.load(f)
        
with open('TranslateY_Dict_GD4.pkl', 'rb') as f:
    TranslateY_Dict_GD4 = pickle.load(f)


#Ploting and maybe saving QualityFunction function
dx0, dy0 = Recall_dx_dy(KnownFrame,UnknownFrame,TranslateX_Dict_GD4,TranslateY_Dict_GD4)

xInches = 7
yInches = 13
ResidualScale = 300

PlotWeightedQualityFunction(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0,xInches,yInches,ResidualScale,SavePlots=False)

In [ ]:
KnownFrame = 'XP2_Y0'
UnknownFrame = 'XP2_YM1'

#Begin by using HelpPickROI function to pick a hopefully valid overlaping ROI
dx0, dy0 = Recall_dx_dy(KnownFrame,UnknownFrame,TranslateX_Dict_GD_Test2,TranslateY_Dict_GD_Test2)

ROI=[50,1750,700,1080]

HelpPickROI(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0)

In [ ]:
#Performing Iterated Gradient Descent
dx, dy, min_mean_weighted_residual = compute_shift_gd_it2(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0,verbose=True)
R_String  = "{:.4e}".format(min_mean_weighted_residual)
print("")
print("---------------------------------------------------------------------------------------")
print("---------------------------------------------------------------------------------------")
print("------------------------------- " + KnownFrame + " and " + UnknownFrame + " -------------------------------")
print("Gradient Descent Starting Point: dx0 = " + str(round(dx0)) + " dy0 = " + str(round(dy0)))
print("Best Gradient Descent Result: dx = " + str(round(dx)) + " dy = " + str(round(dy)))
print("Mean Per Pixel Error Weighted Residual in ROI: " + R_String)
print("---------------------------------------------------------------------------------------")
print("---------------------------------------------------------------------------------------")
print("")

In [ ]:
#Updating and saving the Translation Dictionaries with the new Translations.
TranslateX0 = TranslateX_Dict_GD4[KnownFrame]
TranslateY0 = TranslateY_Dict_GD4[KnownFrame]

TranslateX_Dict_GD4.update({UnknownFrame:int(TranslateX0+dx)})
TranslateY_Dict_GD4.update({UnknownFrame:int(TranslateY0+dy)})

with open('TranslateX_Dict_GD4.pkl', 'wb') as f:
    pickle.dump(TranslateX_Dict_GD4, f)
        
with open('TranslateY_Dict_GD4.pkl', 'wb') as f:
    pickle.dump(TranslateY_Dict_GD4, f)

In [ ]:
#Loading in the new updated and saved Translation Dictionaries 
with open('TranslateX_Dict_GD4.pkl', 'rb') as f:
    TranslateX_Dict_GD4 = pickle.load(f)
        
with open('TranslateY_Dict_GD4.pkl', 'rb') as f:
    TranslateY_Dict_GD4 = pickle.load(f)


#Ploting and maybe saving QualityFunction function
dx0, dy0 = Recall_dx_dy(KnownFrame,UnknownFrame,TranslateX_Dict_GD4,TranslateY_Dict_GD4)

xInches = 7
yInches = 13
ResidualScale = 300

PlotWeightedQualityFunction(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0,xInches,yInches,ResidualScale,SavePlots=False)

In [ ]:
KnownFrame = 'XM2_Y0'
UnknownFrame = 'XM2_YM1'

#Begin by using HelpPickROI function to pick a hopefully valid overlaping ROI
dx0, dy0 = Recall_dx_dy(KnownFrame,UnknownFrame,TranslateX_Dict_GD_Test2,TranslateY_Dict_GD_Test2)

ROI=[50,1700,720,1049]

HelpPickROI(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0)

In [ ]:
#Performing Iterated Gradient Descent
dx, dy, min_mean_weighted_residual = compute_shift_gd_it2(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0,verbose=True)
R_String  = "{:.4e}".format(min_mean_weighted_residual)
print("")
print("---------------------------------------------------------------------------------------")
print("---------------------------------------------------------------------------------------")
print("------------------------------- " + KnownFrame + " and " + UnknownFrame + " -------------------------------")
print("Gradient Descent Starting Point: dx0 = " + str(round(dx0)) + " dy0 = " + str(round(dy0)))
print("Best Gradient Descent Result: dx = " + str(round(dx)) + " dy = " + str(round(dy)))
print("Mean Per Pixel Error Weighted Residual in ROI: " + R_String)
print("---------------------------------------------------------------------------------------")
print("---------------------------------------------------------------------------------------")
print("")

In [ ]:
#Updating and saving the Translation Dictionaries with the new Translations.
TranslateX0 = TranslateX_Dict_GD4[KnownFrame]
TranslateY0 = TranslateY_Dict_GD4[KnownFrame]

TranslateX_Dict_GD4.update({UnknownFrame:int(TranslateX0+dx)})
TranslateY_Dict_GD4.update({UnknownFrame:int(TranslateY0+dy)})

with open('TranslateX_Dict_GD4.pkl', 'wb') as f:
    pickle.dump(TranslateX_Dict_GD4, f)
        
with open('TranslateY_Dict_GD4.pkl', 'wb') as f:
    pickle.dump(TranslateY_Dict_GD4, f)

In [ ]:
#Loading in the new updated and saved Translation Dictionaries 
with open('TranslateX_Dict_GD4.pkl', 'rb') as f:
    TranslateX_Dict_GD4 = pickle.load(f)
        
with open('TranslateY_Dict_GD4.pkl', 'rb') as f:
    TranslateY_Dict_GD4 = pickle.load(f)


#Ploting and maybe saving QualityFunction function
dx0, dy0 = Recall_dx_dy(KnownFrame,UnknownFrame,TranslateX_Dict_GD4,TranslateY_Dict_GD4)

xInches = 13
yInches = 7
ResidualScale = 300

PlotWeightedQualityFunction(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0,xInches,yInches,ResidualScale,SavePlots=False)

In [ ]:
KnownFrame = 'XP2_Y0'
UnknownFrame = 'XP2_YP1'

#Begin by using HelpPickROI function to pick a hopefully valid overlaping ROI
dx0, dy0 = Recall_dx_dy(KnownFrame,UnknownFrame,TranslateX_Dict_GD_Test2,TranslateY_Dict_GD_Test2)

ROI=[50,1680,50,400]

HelpPickROI(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0)

In [ ]:
#Performing Iterated Gradient Descent
dx, dy, min_mean_weighted_residual = compute_shift_gd_it2(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0,verbose=True)
R_String  = "{:.4e}".format(min_mean_weighted_residual)
print("")
print("---------------------------------------------------------------------------------------")
print("---------------------------------------------------------------------------------------")
print("------------------------------- " + KnownFrame + " and " + UnknownFrame + " -------------------------------")
print("Gradient Descent Starting Point: dx0 = " + str(round(dx0)) + " dy0 = " + str(round(dy0)))
print("Best Gradient Descent Result: dx = " + str(round(dx)) + " dy = " + str(round(dy)))
print("Mean Per Pixel Error Weighted Residual in ROI: " + R_String)
print("---------------------------------------------------------------------------------------")
print("---------------------------------------------------------------------------------------")
print("")

In [ ]:
#Updating and saving the Translation Dictionaries with the new Translations.
TranslateX0 = TranslateX_Dict_GD4[KnownFrame]
TranslateY0 = TranslateY_Dict_GD4[KnownFrame]

TranslateX_Dict_GD4.update({UnknownFrame:int(TranslateX0+dx)})
TranslateY_Dict_GD4.update({UnknownFrame:int(TranslateY0+dy)})

with open('TranslateX_Dict_GD4.pkl', 'wb') as f:
    pickle.dump(TranslateX_Dict_GD4, f)
        
with open('TranslateY_Dict_GD4.pkl', 'wb') as f:
    pickle.dump(TranslateY_Dict_GD4, f)

In [ ]:
#Loading in the new updated and saved Translation Dictionaries 
with open('TranslateX_Dict_GD4.pkl', 'rb') as f:
    TranslateX_Dict_GD4 = pickle.load(f)
        
with open('TranslateY_Dict_GD4.pkl', 'rb') as f:
    TranslateY_Dict_GD4 = pickle.load(f)


#Ploting and maybe saving QualityFunction function
dx0, dy0 = Recall_dx_dy(KnownFrame,UnknownFrame,TranslateX_Dict_GD4,TranslateY_Dict_GD4)

xInches = 7
yInches = 13
ResidualScale = 300

PlotWeightedQualityFunction(KnownFrame,UnknownFrame,ROI,HotsArray,dx0,dy0,xInches,yInches,ResidualScale,SavePlots=False)

In [ ]:
#Plot the Mosaic

#loading GD dictionary named as test dictionary to avoid having to relablel everywhere
with open('TranslateX_Dict_GD4.pkl', 'rb') as f:
    TranslateX_Dict_Test = pickle.load(f)
        
with open('TranslateY_Dict_GD4.pkl', 'rb') as f:
    TranslateY_Dict_Test = pickle.load(f)
    
#initalizing mosaic
Mosaic = np.zeros((7*DimY,4*DimX))

for SetName in tqdm(SetNamesList):
    MeanFluxFrame = FluxFrameDict[SetName]
    
    Coords = -1*CoordsDict[SetName]
    
    MosaicX = Coords[0]
    MosaicY = Coords[1]
    
    if SetName in TranslateX_Dict_Test:
        TranslateX = TranslateX_Dict_Test[SetName]
        TranslateY = TranslateY_Dict_Test[SetName]
    else:
        TranslateX = int(((MosaicX/2) + 2)*DimX)
        TranslateY = int((MosaicY + 3)*DimY)
    
    for xi in range(DimX):
        for yi in range(DimY):
            NewXi = xi + TranslateX
            NewYi = yi + TranslateY
            
            Mosaic[NewYi,NewXi] = MeanFluxFrame[yi,xi]

            
SMALL_SIZE = 15
MEDIUM_SIZE = 15
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
            
f, ax = plt.subplots(1, 1)
im1 = ax.imshow(Mosaic, cmap='turbo', vmax=2500)
divider = make_axes_locatable(ax)
cax1 = divider.append_axes("right", size="5%", pad=0.15)
cbar1 = plt.colorbar(im1, cax=cax1)
cbar1.formatter.set_powerlimits((0, 0))
#cbar1.formatter.set_useMathText(True)
for SetName in SetNamesList:
    Coords = -1*CoordsDict[SetName]
    
    MosaicX = Coords[0]
    MosaicY = Coords[1]
    
    if SetName in TranslateX_Dict_Test:
        TranslateX = TranslateX_Dict_Test[SetName]
        TranslateY = TranslateY_Dict_Test[SetName]
    else:
        TranslateX = int(((MosaicX/2) + 2)*DimX)
        TranslateY = int((MosaicY + 3)*DimY)
            
    ROI = [TranslateX, TranslateX + DimX -1, TranslateY, TranslateY + DimY -1]
    xROI, yROI = ROItoPoints(ROI)

    ax.fill(xROI, yROI, facecolor='none', edgecolor='magenta', linewidth=2, label = "ROI")
ax.set_xlabel('Pixel x')
ax.set_ylabel('Pixel y')
ax.set_title(r"Mean Flux Mosaic [$e^{-}/s$]"
               "\n" r"490.5 eV", loc = 'center')

plt.subplots_adjust(hspace = 0.35)
figure = plt.gcf()
#figure.set_size_inches(13, 13)
figure.set_size_inches(10, 10)
SaveString = "Mean_Flux_Mosaic_A.png"
#plt.savefig(SaveString, dpi=300,bbox_inches='tight')
#plt.savefig(SaveString, dpi=750,bbox_inches='tight')
plt.show()

#Clipped plot

f, ax = plt.subplots(1, 1)
im1 = ax.imshow(Mosaic, cmap='turbo', vmin = 0, vmax = 20) #vmax used to be 2, 9 is good too
divider = make_axes_locatable(ax)
cax1 = divider.append_axes("right", size="5%", pad=0.15)
cbar1 = plt.colorbar(im1, cax=cax1)
cbar1.formatter.set_powerlimits((0, 0))
#cbar1.formatter.set_useMathText(True)
for SetName in SetNamesList:
    Coords = -1*CoordsDict[SetName]
    
    MosaicX = Coords[0]
    MosaicY = Coords[1]
    
    if SetName in TranslateX_Dict_Test:
        TranslateX = TranslateX_Dict_Test[SetName]
        TranslateY = TranslateY_Dict_Test[SetName]
    else:
        TranslateX = int(((MosaicX/2) + 2)*DimX)
        TranslateY = int((MosaicY + 3)*DimY)
            
    ROI = [TranslateX, TranslateX + DimX -1, TranslateY, TranslateY + DimY -1]
    xROI, yROI = ROItoPoints(ROI)

    ax.fill(xROI, yROI, facecolor='none', edgecolor='magenta', linewidth=2, label = "ROI")
ax.set_xlabel('Pixel x')
ax.set_ylabel('Pixel y')
ax.set_title(r"Mean Flux Mosaic [$e^{-}/s$]"
               "\n" r"490.5 eV", loc = 'center')

plt.subplots_adjust(hspace = 0.35)
figure = plt.gcf()
#figure.set_size_inches(13, 13)
figure.set_size_inches(10, 10)
SaveString = "Mean_Flux_Mosaic_B.png"
#plt.savefig(SaveString, dpi=300,bbox_inches='tight')
#plt.savefig(SaveString, dpi=750,bbox_inches='tight')
plt.show()

#unclipped no boxes
f, ax = plt.subplots(1, 1)
im1 = ax.imshow(Mosaic, cmap='turbo', vmax=2500)
divider = make_axes_locatable(ax)
cax1 = divider.append_axes("right", size="5%", pad=0.15)
cbar1 = plt.colorbar(im1, cax=cax1)
cbar1.formatter.set_powerlimits((0, 0))
ax.set_xlabel('Pixel x')
ax.set_ylabel('Pixel y')
ax.set_title(r"Mean Flux Mosaic [$e^{-}/s$]"
               "\n" r"490.5 eV", loc = 'center')

plt.subplots_adjust(hspace = 0.35)
figure = plt.gcf()
#figure.set_size_inches(13, 13)
figure.set_size_inches(10, 10)
SaveString = "Mean_Flux_Mosaic_C.png"
#plt.savefig(SaveString, dpi=300,bbox_inches='tight')
#plt.savefig(SaveString, dpi=750,bbox_inches='tight')
plt.show()

#Clipped plot, no boxes

f, ax = plt.subplots(1, 1)
im1 = ax.imshow(Mosaic, cmap='turbo', vmin = 0, vmax = 20) #vmax used to be 2
divider = make_axes_locatable(ax)
cax1 = divider.append_axes("right", size="5%", pad=0.15)
cbar1 = plt.colorbar(im1, cax=cax1)
cbar1.formatter.set_powerlimits((0, 0))
#cbar1.formatter.set_useMathText(True)
ax.set_xlabel('Pixel x')
ax.set_ylabel('Pixel y')
ax.set_title(r"Mean Flux Mosaic [$e^{-}/s$]"
               "\n" r"490.5 eV", loc = 'center')

plt.subplots_adjust(hspace = 0.35)
figure = plt.gcf()
#figure.set_size_inches(13, 13)
figure.set_size_inches(10, 10)
SaveString = "Mean_Flux_Mosaic_D.png"
#plt.savefig(SaveString, dpi=300,bbox_inches='tight')
#plt.savefig(SaveString, dpi=750,bbox_inches='tight')
plt.show()

In [ ]:
ROI1=[250,1750,710,1050]
dx1 = 155
dy1 = 686
compute_shift_gd3(KnownFrame,UnknownFrame,ROI1,HotsArray,dx1,dy1,3,verbose=True)